# Example of using Agent node

In [1]:
import os
from dotenv import load_dotenv
from aic_flow.nodes.ai import Agent
from aic_flow.nodes.telegram import MessageTelegram


load_dotenv()


model_config = {
    "model": "gpt-4o-mini",
    "api_key": os.getenv("OPENAI_API_KEY"),
}

/Users/s.jiang/Development/aic-flow/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "schema" in "StructuredOutput" shadows an attribute in parent "BaseModel"
  warnings.warn(


## Structured output with vanilla JSON dict

In [2]:
so_schema = """
json_dict = {
    "type": "object",
    "title": "Person",
    "description": "A person",
    "properties": {
        "name": {"type": "string"},
    },
    "required": ["name"],
}
"""

so_config = {
    "type": "json_dict",
    "schema": so_schema,
}

agent_node = Agent(
    name="agent",
    model_config=model_config,
    structured_output=so_config,
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)
config = {"configurable": {"thread_id": "123"}}
result = await agent_node(
    {"messages": [{"role": "user", "content": "What's your name?"}]}, config
)

result["structured_response"]

{'name': 'John Doe'}

## Structured output with OpenAI JSON dict

In [3]:
so_schema = """
oai_json_schema = {
    "name": "get_person",
    "strict": True,
    "schema": {
        "type": "object",
        "properties": {"name": {"type": "string"}},
        "additionalProperties": False,
        "required": ["name"],
    },
}
"""

so_config = {
    "type": "json_dict",
    "schema": so_schema,
}

agent_node = Agent(
    name="agent",
    model_config=model_config,
    structured_output=so_config,
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)
config = {"configurable": {"thread_id": "123"}}
result = await agent_node(
    {"messages": [{"role": "user", "content": "What's your name?"}]}, config
)

result["structured_response"]

{'name': 'John Doe'}

## Structured output with Pydantic Models

In [4]:
so_schema = """
class Person(BaseModel):
    \"\"\"A Person.\"\"\"

    name: str
"""

so_config = {
    "type": "json_dict",
    "schema": so_schema,
}

agent_node = Agent(
    name="agent",
    model_config=model_config,
    structured_output=so_config,
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)
config = {"configurable": {"thread_id": "123"}}
result = await agent_node(
    {"messages": [{"role": "user", "content": "What's your name?"}]}, config
)

result["structured_response"]

Person(name='John Doe')

## Structured output with Typed dict

In [5]:
so_schema = """
class Person(TypedDict):
    \"\"\"A Person.\"\"\"

    name: str
"""

so_config = {
    "type": "json_dict",
    "schema": so_schema,
}

agent_node = Agent(
    name="agent",
    model_config=model_config,
    structured_output=so_config,
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)
config = {"configurable": {"thread_id": "123"}}
result = await agent_node(
    {"messages": [{"role": "user", "content": "What's your name?"}]}, config
)

result["structured_response"]

{'name': 'John Doe'}

## Use a node as a tool

Main differences of a graph node and a function tool: A graph node only receives `state: dict` as input, and returns a dict, while a function tool can have arbitrary inputs.

In [6]:
from pprint import pprint


telegram_node = MessageTelegram(
    name="MessageTelegram",
    token=os.getenv("TELEGRAM_TOKEN"),
)

agent_node = Agent(
    name="agent",
    model_config=model_config,
    tools=[telegram_node.tool_run],
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)

config = {"configurable": {"thread_id": "123"}}
result = await agent_node(
    {
        "messages": [
            {"role": "user", "content": f"Say hello to {os.getenv('TELEGRAM_CHAT_ID')}"}
        ]
    },
    config,
)

pprint(result["outputs"])

{'agent': {'messages': [HumanMessage(content='Say hello to None', additional_kwargs={}, response_metadata={}, id='3a03f0e2-d8da-42ac-83c8-c8a350b6a40f'),
                        AIMessage(content='Hello! If "None" refers to someone specific or if it\'s just a placeholder, I\'m here to help. Let me know if you need anything!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 103, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_92e0377081', 'id': 'chatcmpl-Ba0Gdzu3mSQXzVyQkAGS2QgkmQysA', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5eb49c8c-2144-4533-a16e-95d47f7ed6c5-0', usage_metadata={'input_tokens': 103, 'output_tokens': 31, 'total_tokens': 134,